In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
df= spark.read \
    .option("header","true") \
        .csv("fhvhv_tripdata_2021-01.csv")

In [ ]:
df.show()

In [ ]:
df.head(5)

In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv("head.csv")
df.dtypes

In [ ]:
spark.createDataFrame(df).schema

In [ ]:
from pyspark.sql import types

In [ ]:
schema = types.StructType(
    [types.StructField('hvfhs_license_num', types.StringType(), True),
     types.StructField('dispatching_base_num', types.StringType(), True),
     types.StructField('pickup_datetime', types.TimestampType(), True),
     types.StructField('dropoff_datetime', types.TimestampType(), True),
      types.StructField('PULocationID', types.LongType(), True),
       types.StructField('DOLocationID', types.LongType(), True),
       types.StructField('SR_Flag', types.StringType(), True)])

In [ ]:
df= spark.read \
    .option("header","true") \
        .schema(schema) \
        .csv("fhvhv_tripdata_2021-01.csv")

In [ ]:
df.head(10)

In [ ]:
df = df.repartition(24)

In [ ]:
df.write.parquet('fhvhv/2021/01')

In [ ]:
df = spark.read.parquet('fhvhv/2021/01/')

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql import functions as F

In [ ]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif  num % 3 == 0:
         return f'a/{num:03x}'
    else : 
        return f'e/{num:03x}'

In [ ]:
crazy_stuff_udf=F.udf(crazy_stuff,returnType=types.StringType())

In [ ]:
df \
.withColumn('pickup_date',F.to_date(df.pickup_datetime)) \
.withColumn('dropoff_date',F.to_date(df.dropoff_datetime)) \
.withColumn('base_id',crazy_stuff_udf(df.dispatching_base_num)) \
.select('base_id','pickup_date','dropoff_date','PULocationID','DOLocationID') \
.show()

In [ ]:
df.select('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003').show()

In [ ]:
!